In [1]:
import torch
import os
import pandas as pd
import numpy as np 
from tqdm import tqdm
from glob import glob

In [2]:
labels_df = pd.read_csv("/home/single1/BACKUP/SamHUyen/mammo/huyen/csv_singleview.csv")
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19276 entries, 0 to 19275
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   patient_id      19276 non-null  object
 1   study_id        19276 non-null  object
 2   series_id       19276 non-null  object
 3   image_id        19276 non-null  object
 4   image_height    19276 non-null  int64 
 5   image_width     19276 non-null  int64 
 6   bbox            3549 non-null   object
 7   blabel          19276 non-null  object
 8   bsession_id     3549 non-null   object
 9   gbirad          19276 non-null  object
 10  gdensity        19276 non-null  object
 11  gscope          9600 non-null   object
 12  glaterality     19276 non-null  object
 13  gview_position  19276 non-null  object
 14  gsession_id     11352 non-null  object
 15  multilabel      19276 non-null  object
 16  fold            19276 non-null  object
dtypes: int64(2), object(15)
memory usage: 2.5+ MB


In [3]:
#path of image
labels_df["path"] = "/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/images/" + labels_df["study_id"] + "/" + labels_df["image_id"] + ".png" 

In [4]:
#CROP IMAGE
#Data processing
miss_img = []
list_img = os.listdir("/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/crop-images/crop_images/")
list_crop_img = []
for img in list_img:
    list_crop_img.append(img.replace(".png",""))
list_img_id = labels_df["image_id"].values
for img_id in tqdm(list_img_id):
    if img_id not in list_crop_img:
        miss_img.append(img_id)

print(len(img_id))

100%|██████████| 19276/19276 [00:01<00:00, 10018.01it/s]55



In [6]:
for i in miss_img:
    print(labels_df[labels_df["image_id"] == i].index)

Int64Index([2438, 18074], dtype='int64')
Int64Index([5642], dtype='int64')
Int64Index([7422], dtype='int64')
Int64Index([9305], dtype='int64')
Int64Index([9310], dtype='int64')
Int64Index([9824], dtype='int64')
Int64Index([10483], dtype='int64')
Int64Index([11724], dtype='int64')
Int64Index([13368], dtype='int64')
Int64Index([2438, 18074], dtype='int64')


In [8]:
#crop id khong co trong folder image
labels_df = labels_df.drop([2438, 18074,5642,7422,9305,9310,9824,10483,11724,13368])

In [4]:
#Training 
#Custom Batch Generator:
import torchvision
from glob import glob
import os
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.utils import make_grid
import time
from torch.utils.data import Dataset
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
#import cv2
%matplotlib inline

In [5]:
#Dataloader
class MultiClassMammo(Dataset):
    
    def __init__(self, dataframe, transform = None):
        
        self.dataframe = dataframe
        self.imagespath = dataframe.path.values
        self.transform = transform
        self.labels = dataframe.multilabel.values.tolist()
        
        
    def __len__(self):
        return len(self.dataframe)
    
    
    def __getitem__(self, index):
        
        image = Image.open(self.imagespath[index]).convert('RGB')
        #image = torch.from_numpy(cv2.imread(self.imagespath[index]))
        #torch.reshape(image, (3,1759,1400))
        #image = cv2.imread(self.imagespath[index])
        label = self.labels[index]
        label = label.strip('][').split(', ')
        label = np.array(label).astype('float32')
        sample = {'image': image, 'label': label}
        if self.transform:
            image = self.transform(sample['image'])
            sample = {'image': image, 'label': label}
        
        return sample

In [6]:
#transform images size from (3518, 2800, 3) to (1759,1400,3)
tfms = transforms.Compose([transforms.Resize((256, 256)),
                           transforms.ToTensor()])



In [7]:
#fold: ['train', 'valid', 'holdout']
train_dl = MultiClassMammo(labels_df[labels_df["fold"]=="train"], transform = tfms) #tfms
val_dl = MultiClassMammo(labels_df[labels_df["fold"]=="valid"], transform = tfms)
#holdout_dl = MultiClassMammo(labels_df[labels_df["fold"]=="holdout"], transform = tfms)


In [8]:
print(train_dl[1]['label'])
print(train_dl[1]['image'].shape)
plt.imshow(torchvision.utils.make_grid(train_dl[1]['image']).permute(1, 2, 0))

[0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
torch.Size([3, 256, 256])


In [10]:
import torch.nn.functional as F
class MultiClassifier(nn.Module):
    def __init__(self):
        super(MultiClassifier, self).__init__()
        self.ConvLayer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3), # 3, 512, 512 #input : 3 channels, output : 64 channels, kernel size: 3
            nn.MaxPool2d(2), # op: 16, 256, 256
            nn.ReLU(), # op: 64, 256, 256
        )
        self.ConvLayer2 = nn.Sequential(
            nn.Conv2d(64, 128, 3), # 64, 256, 256   
            nn.MaxPool2d(2), #op: 128, 127, 127
            nn.ReLU() # op: 128, 127, 127
        )
        self.ConvLayer3 = nn.Sequential(
            nn.Conv2d(128, 256, 3), # 128, 127, 127
            nn.MaxPool2d(2), #op: 256, 63, 63
            nn.ReLU() #op: 256, 63, 63
        )
        self.ConvLayer4 = nn.Sequential(
            nn.Conv2d(256, 512, 3), # 256, 63, 63
            nn.MaxPool2d(2), #op: 512, 30, 30
            nn.ReLU(), #op: 512, 30, 30
            nn.Dropout(0.2)
        )
        self.Linear1 = nn.Linear(512 * 30 * 30, 1024) #512 * 14 * 14
        self.Linear2 = nn.Linear(1024, 256) #number of nodes input, number of node output
        self.Linear3 = nn.Linear(256, 10)
        
        
    def forward(self, x):
        x = self.ConvLayer1(x)
        x = self.ConvLayer2(x)
        x = self.ConvLayer3(x)
        x = self.ConvLayer4(x)
        x = x.view(x.size(0), -1)
        x = self.Linear1(x)
        x = self.Linear2(x)
        x = self.Linear3(x)
        return F.sigmoid(x)

In [11]:
#check cuda
def check_cuda():
    _cuda = False
    if torch.cuda.is_available():
        _cuda = True
    return _cuda

In [12]:
is_cuda = check_cuda()

In [13]:
#model = MultiClassifier()
#Pretrain
model = torch.load("/home/single1/BACKUP/SamHUyen/mammo/sam/singleview_sam/model/model_epoch.pt", map_location=torch.device('cuda:0'))

if is_cuda:
    model.cuda()

In [14]:

train_dataloader = torch.utils.data.DataLoader(train_dl, shuffle = True, batch_size = 2, num_workers = 3)
valid_dataloader = torch.utils.data.DataLoader(val_dl, shuffle = True, batch_size = 2, num_workers = 3)

#hold_dataloader = torch.utils.data.DataLoader(holdout_dl, shuffle = True, batch_size = 2, num_workers = 3)

In [35]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0001, momentum = 0.9)

In [33]:
from pprint import pprint
def fit_model(epochs, model, dataloader, phase = 'training', volatile = False):
    
    pprint("Epoch: {}".format(epochs))

    if phase == 'training':
        model.train()
        
    if phase == 'validataion':
        model.eval()
        volatile = True
        
    running_loss = []
    running_acc = []
    b = 0
    for i, data in enumerate(dataloader):
        

        inputs, target = data['image'].cuda(), data['label'].cuda() 
        
        inputs, target = Variable(inputs), Variable(target)
        
        if phase == 'training':
            optimizer.zero_grad()
        
        #print(inputs)
        ops = model(inputs)
        
        acc_ = []
        for i, d in enumerate(ops, 0):
           
            acc = pred_acc(torch.Tensor.cpu(target[i]), torch.Tensor.cpu(d))
            #print("True Value: ", torch.Tensor.cpu(target[i]))
            #print("Predict value: ", torch.round(torch.Tensor.cpu(d)))
            #print(acc)
            acc_.append(acc)
        #print(ops.dtype)
        #print(target)
        loss = criterion(ops, target)
                
        running_loss.append(loss.item())
        running_acc.append(np.sum(acc_)/len(acc_))
        b += 1
       
        if phase == 'training':
            
            loss.backward()
        
            optimizer.step()
            
    total_batch_loss = np.asarray(running_loss).mean()
    total_batch_acc = np.asarray(running_acc).mean()
    

    pprint("{} loss is {} ".format(phase,total_batch_loss))
    pprint("{} accuracy is {} ".format(phase, total_batch_acc))
    
    return total_batch_loss, total_batch_acc

In [31]:
def pred_acc(original, predicted):
    if torch.all(original.eq(torch.round(predicted))):
        return 1
    else:
        return 0
    #return torch.round(predicted==original).sum().numpy()/len(original)

In [36]:
#TRaining model
trn_losses = []; trn_acc = []
val_losses = []; val_acc = []
for i in tqdm(range(0, 5)):
    trn_l, trn_a = fit_model(i, model, train_dataloader)
    val_l, val_a = fit_model(i, model, valid_dataloader, phase = 'validation')
    trn_losses.append(trn_l); trn_acc.append(trn_a)
    val_losses.append(val_l); val_acc.append(val_a)


  0%|          | 0/5 [00:00<?, ?it/s]'Epoch: 0'
'training loss is 0.22100342250255645 '
'training accuracy is 0.5490065243179122 '
'Epoch: 0'
 20%|██        | 1/5 [07:29<29:58, 449.62s/it]'validation loss is 0.23844439386444327 '
'validation accuracy is 0.5259695290858726 '
'Epoch: 1'
'training loss is 0.21992019719575684 '
'training accuracy is 0.5474495848161328 '
'Epoch: 1'
 40%|████      | 2/5 [15:00<22:31, 450.62s/it]'validation loss is 0.23779170957092102 '
'validation accuracy is 0.5290858725761773 '
'Epoch: 2'
'training loss is 0.21933273657822755 '
'training accuracy is 0.5464857651245552 '
'Epoch: 2'
 60%|██████    | 3/5 [22:34<15:03, 451.76s/it]'validation loss is 0.23816868591888543 '
'validation accuracy is 0.5169667590027701 '
'Epoch: 3'
'training loss is 0.21885710482873366 '
'training accuracy is 0.5431494661921709 '
'Epoch: 3'
 80%|████████  | 4/5 [30:11<07:34, 454.01s/it]'validation loss is 0.23754921169832885 '
'validation accuracy is 0.5259695290858726 '
'Epoch: 4'


In [17]:
#Test
model = torch.load("/home/single1/BACKUP/SamHUyen/mammo/sam/singleview_sam/model/model_epoch.pt", map_location=torch.device('cuda:0'))
model = model.eval()

val_loss, val_acc = fit_model(0, model, valid_dataloader, phase = 'validation')

'Epoch: 0'
'validation loss is 0.2507135372020696 '
'validation accuracy is 0.8989265927977841 '


In [ ]:
torch.save("/home/single1/BACKUP/SamHUyen/mammo/sam/singleview_sam/model/model_imgcrop.pt",model)